# FineTuning a Pretrained Model to Distinguish Between Bees and Ants

In this last but one section we will see how to perform the **fine tuning** of a pretrained model (for real, this time).
To do this we will use a simpler dataset, composed by images belonging to just 2 classes, bees and ants.

Finetuning means that we download and use a pretrained model as in the previous section, but instead of just training the newly added last layer, we should re-train *all* the model's parameters, but using a very low learning rate, in this way we won't change the parameters as much, but we'll permit our model to slowly adapt the parameters to the new task, thus the term FineTuning.

Let's add our usual imports

In [ ]:
import torch
from torch.utils import data

import numpy as np
from tqdm import tqdm


from torchvision.datasets.mnist import FashionMNIST
from torchvision.models.resnet import resnet18
from torchvision import transforms, utils, datasets

from torch import nn
from torch import optim
from torch.nn import functional as F

import os
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

let's create our dataset objects

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform_tr = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

data_dir = '../data/hymenoptera_data'


In [ ]:
image_datasets_tr = datasets.ImageFolder(
    os.path.join(data_dir, 'train'), 
    transform_tr)
image_datasets_val = datasets.ImageFolder(
    os.path.join(data_dir, 'val'), 
    transform_val)

dataloader_tr = data.DataLoader(image_datasets_tr, shuffle=True, batch_size=4, num_workers=4)
dataloader_val = data.DataLoader(image_datasets_val, shuffle=False, batch_size=4, num_workers=4)

dataset_sizes = {'train': len(image_datasets_tr), 'val': len(image_datasets_val)}

class_names = image_datasets_tr.classes

let's show some sample from the dataset and its label, for this purpose we will create a simple function

In [ ]:
def imshow(inp, title=None):
    '''imshow for Tensor'''
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.array(std) * inp + np.array(mean)
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=[10,10])
    plt.imshow(inp)
    if title:
        plt.title(title)
    plt.pause(0.001)

In [ ]:
inputs, classes = next(iter(dataloader_tr))
out = utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

Bees, and also Ants.

Let's get our pretrained model, replace the pooling layer and the last fc layer, we will also initialize the newly created fc layer weight data using a normal function having mu=0 and var=0.001

In [ ]:
model = resnet18(pretrained=True)
# replace the avgpool (it's a 7x7 pooling expecting Cx7x7 tensors)
model.avgpool = nn.AdaptiveAvgPool2d((1,1))
# replace the last layer (it has 1000 out features and we need new weights)
model.fc = nn.Linear(model.fc.in_features, 2)
model.fc.weight.data.normal_(0.0, 0.001)

we use a CrossEntropyLoss and SGD algorithm for classification as usual for classification problems. This time we will also make use of a *Learning Rate Scheduler*, which permits us to update the learning rate following a certain rule, in this case we will use the simple StepLR scheduler which accept a **step_size** parameter which corresponds to the amount of steps between each lr update, and a gamma parameter, which will get multiplied with our LR and the results will be our new LR.

in brief the StepLR does:

    every step_size steps:
        LR *= gamma

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(
    optimizer, 
    step_size=7, 
    gamma=0.1)

Let's get our training started, as we don't want to wait and want to go drink some beer, we will train the model for just 1 epoch and evaluate the model on the validation dataset

In [ ]:
for epoch in range(1):

    for i, (x, y) in enumerate(dataloader_tr):
        l = loss(model(x), y)

        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        
        print('Epoch: {}, batch_idx: {}/{}, loss: {}'.format(
            epoch, i, len(dataloader_tr)-1, l.item()))

Q: Why this time the pretrained model is training faster??

because the dataset is much more small, in terms of number of samples, even if the images are much larger, we roughly have almost a 10% of its number of samples, thus the faster epochs

Let's evaluate the model on the validation dataset as usual

In [ ]:
model.eval()
preds = []
ys = []
for x, y in dataloader_val:
    preds.extend(model(x).max(1)[1].data.tolist())
    ys.extend(y.data)

corrects = (np.array(preds) == np.array(ys))
print('Accuracy: {}'.format(corrects.mean()))

that's actually an awesome result with just one epoch of training. But let's count how many images the model classifies wrongly

In [ ]:
print('correct predicitons: {}'.format(
    np.sum(np.array(preds) == np.array(ys))))

print('wrong predicitons: {}'.format(
    np.sum(np.array(preds) != np.array(ys))))